In [1]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from ray.tune.search.optuna import OptunaSearch
from ray import tune, train
import ray
from Opening_Dataset import X, y

In [2]:
ray.init()

2024-01-18 16:39:13,991	INFO worker.py:1724 -- Started a local Ray instance.


Python version:,3.10.13
Ray version:,2.9.0


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
search_space = {
    "hidden_layer_sizes": tune.choice([(30, 128, 5), (30, 256, 5), (30, 68, 5)]),
    "activation": tune.choice(['tanh', 'relu']),
    "alpha": tune.loguniform(1e-6, 1e-3),
    "learning_rate_init": tune.loguniform(1e-5, 1e-2),
    "max_iter": tune.choice([200, 300, 500]),  # You can adjust this value
}

In [5]:
def train_func(config):
    clf = MLPClassifier(
        **config,
        verbose=False,
        random_state=42,
        early_stopping=True,
        n_iter_no_change=10
    )
    clf.fit(X_train, y_train)
    train.report({
        "mean_accuracy": clf.score(X_test, y_test),
        "train_loss": clf.loss_,
    })

In [6]:
algo = OptunaSearch()
tune_config = tune.TuneConfig(
    metric="mean_accuracy",
    mode="max",
    num_samples=10,
    search_alg=algo,    
)

In [7]:
tuner = tune.Tuner(
    train_func,
    tune_config=tune_config,
    param_space=search_space,
)

In [8]:
tuner.fit()

(raylet) Warning: The actor ImplicitFunc is very large (53 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


2024-01-18 16:39:54,139	INFO tensorboardx.py:275 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layer_sizes': (30, 68, 5)}
2024-01-18 16:40:16,697	INFO tensorboardx.py:275 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layer_sizes': (30, 128, 5)}
2024-01-18 16:40:17,716	INFO tensorboardx.py:275 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layer_sizes': (30, 68, 5)}
2024-01-18 16:40:18,688	INFO tensorboardx.py:275 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layer_sizes': (30, 256, 5)}
2024-01-18 16:40:20,760	INFO tensorboardx.py:275 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layer_sizes': (30, 128, 5)}
2024-01-18 16:40:28,764	INFO tensorboardx.py:275 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layer_sizes': (30, 68, 5)}
2024-01-18 16:40:30,935	INFO te

ResultGrid<[
  Result(
    metrics={'mean_accuracy': 0.7668249703180654, 'train_loss': 0.41639528207123205},
    path='/home/ayakovyna/ray_results/train_func_2024-01-18_16-39-15/train_func_4b6ae59c_1_activation=tanh,alpha=0.0000,hidden_layer_sizes=30_128_5,learning_rate_init=0.0006,max_iter=500_2024-01-18_16-39-19',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    metrics={'mean_accuracy': 0.7701368493407487, 'train_loss': 0.40138258857015485},
    path='/home/ayakovyna/ray_results/train_func_2024-01-18_16-39-15/train_func_91fe1251_2_activation=relu,alpha=0.0000,hidden_layer_sizes=30_256_5,learning_rate_init=0.0008,max_iter=500_2024-01-18_16-39-24',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    metrics={'mean_accuracy': 0.7665437730425545, 'train_loss': 0.4253233497433436},
    path='/home/ayakovyna/ray_results/train_func_2024-01-18_16-39-15/train_func_a349805c_3_activation=relu,alpha=0.0001,hidden_layer_sizes=30_68_5,learning_rate_init=0.0029,max_iter=30